# Student Performance 🤓📊

Aluno: <span style="color:red">Luciano Felix Dias</span>

Entrega: 16 de maio de 2024


# Inicio do projeto

## Leitura dos dados e correção de tipos de dados

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
DATA_DIR = Path.cwd() / "data"
DATA_DIR

In [ ]:
COLUMNS_TYPES = {
    "category": [
        "school",
        "sex",
        "address",
        "famsize",
        "Pstatus",
        "Mjob",
        "Fjob",
        "reason",
        "guardian",
        "schoolsup",
        "famsup",
        "paid",
        "activities",
        "nursery",
        "higher",
        "internet",
        "romantic",
    ],
    "float64": [
        "Medu",
        "Fedu",
        "traveltime",
        "studytime",
        "failures",
        "famrel",
        "freetime",
        "goout",
        "Dalc",
        "Walc",
        "health",
    ],
    "int64": [
        "age",
        "absences",
        "grade",
    ],
}


def load_data(filepath: Path) -> pd.DataFrame:
    df = pd.read_csv(filepath)

    format_data(df, COLUMNS_TYPES)

    return split_features_target(df, "grade")


def format_data(df: pd.DataFrame, format):
    for column_type, column_group in format.items():
        for column in column_group:
            df[column] = df[column].astype(column_type)


def split_features_target(df: pd.DataFrame, targets_column: str):
    target = df[targets_column].copy()

    df.drop(columns=[targets_column], inplace=True)

    return df, target

In [ ]:
X, y = load_data(DATA_DIR / "student_performance.csv")
X

In [ ]:
y

## Visualização simples dos dados

Chamar isso de análise exploratória é vexatório...

In [ ]:
def make_barplots(X: pd.DataFrame, y: pd.Series) -> None:
    fig, axes = plt.subplots(10, 3, figsize=(12, 45))
    axes = axes.flatten()

    fig.tight_layout(h_pad=8, w_pad=5)

    for index, (column_name, column_series) in enumerate(X.items()):
        column_series \
            .value_counts() \
            .sort_index(ascending=False) \
            .plot \
            .barh(ax=axes[index])
        axes[index].set_title(column_name)

    fig.suptitle("Barplots of Categorical and Ordinal Features", fontsize=16)
    fig.subplots_adjust(top=0.96)
    plt.show()

    plt.figure(figsize=(3, 4))
    y.plot.hist(bins=20, edgecolor="black")
    plt.title("Histogram of Target")
    plt.show()
    

In [ ]:
def make_boxplots(X: pd.DataFrame, y: pd.Series) -> None:
    df = pd.concat([X, y], axis=1)

    fig, axes = plt.subplots(10, 3, figsize=(12, 45))
    axes = axes.flatten()

    fig.tight_layout(h_pad=8, w_pad=5)

    for index, column in enumerate(X.columns):
        ax = axes[index]
        bp_objs = df.boxplot(
            by=["school", column],
            column="grade",
            ax=ax,
            rot=45,
            fontsize=8,
            return_type="dict",
            patch_artist=True,
        )
        boxes = bp_objs["grade"]["boxes"]
        box_colors = [
            "lightgreen" if "(GP," in tick.get_text() else "lightblue"
            for tick in ax.get_xticklabels()
        ]
        for box, color in zip(boxes, box_colors):
            box.set_facecolor(color)
        ax.set_title(column)
        ax.set_xlabel("")
    fig.suptitle("Grade distribution by feature and school", fontsize=16)
    fig.subplots_adjust(top=0.96)
    plt.show()

In [ ]:
make_barplots(X, y)
make_boxplots(X, y)

## Separação treino-teste e modelagem inicial

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [ ]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.dummy import DummyRegressor

In [ ]:
categorical_features = X.select_dtypes(include=["category"]).columns
numerical_features = X.select_dtypes(include=["float64", "int64"]).columns

num_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("power_transformer", PowerTransformer()),
])

cat_pipeline = Pipeline([
    ("encoder", OneHotEncoder(drop="first")),
])

preprocessing_pipeline = ColumnTransformer(
    transformers=[
        ("num", num_pipeline, numerical_features),
        ("cat", cat_pipeline, categorical_features),
    ],
    remainder="passthrough",
)

pipe = Pipeline([
    ("preprocessor", preprocessing_pipeline),
    ("regressor", DummyRegressor(strategy="mean")),
])

pipe

In [ ]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.linear_model import Ridge, HuberRegressor

# Quanto maior o numero de splits, maior a significância estatística da
# validação cruzada, mas também maior o tempo de execução.
num_splits = 1_000

param_grid = [
    {
        "regressor": [DummyRegressor(strategy="mean")],
    }, {
        "regressor": [Ridge()],
        "regressor__alpha": np.logspace(-3, 3, 7),
    }, {
        "regressor": [HuberRegressor()],
        "regressor__alpha": [0.0001, 0.001, 0.01],
        "regressor__epsilon": [1.0, 1.5, 2.0],
    },
]

test_fraction = 0.2
num_samples_total = len(y_train)
num_samples_test = int(test_fraction * num_samples_total)
num_samples_train = num_samples_total - num_samples_test

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=ShuffleSplit(
        n_splits=num_splits,
        test_size=num_samples_test,
        random_state=42,
    ),
    n_jobs=-1,
    scoring="neg_root_mean_squared_error",
)

grid.fit(X_train, y_train)

In [ ]:
results_df = pd.DataFrame(grid.cv_results_) \
    .sort_values(by='rank_test_score')

results_df = results_df \
    .set_index(
        results_df["params"] \
            .apply(lambda x: "_".join(str(val) for val in x.values()))
    ) \
    .rename_axis("model")

model_scores = results_df.filter(regex=r"split\d*_test_score")

model_scores

In [ ]:
mean_perf = model_scores.agg(["mean", "std"], axis=1)
mean_perf["std"] = mean_perf["std"] / np.sqrt(num_splits)
mean_perf = mean_perf.sort_values("mean", ascending=False)
mean_perf

## Comparação de modelos

O código a seguir foi copiado de https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_stats.html

In [ ]:
import numpy as np

from scipy.stats import t


def corrected_std(differences, n_train, n_test):
    """Corrects standard deviation using Nadeau and Bengio's approach.

    Parameters
    ----------
    differences : ndarray of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    corrected_std : float
        Variance-corrected standard deviation of the set of differences.
    """
    # kr = k times r, r times repeated k-fold crossvalidation,
    # kr equals the number of times the model was evaluated
    kr = len(differences)
    corrected_var = np.var(differences, ddof=1) * (1 / kr + n_test / n_train)
    corrected_std = np.sqrt(corrected_var)
    return corrected_std


def compute_corrected_ttest(differences, df, n_train, n_test):
    """Computes right-tailed paired t-test with corrected variance.

    Parameters
    ----------
    differences : array-like of shape (n_samples,)
        Vector containing the differences in the score metrics of two models.
    df : int
        Degrees of freedom.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    t_stat : float
        Variance-corrected t-statistic.
    p_val : float
        Variance-corrected p-value.
    """
    mean = np.mean(differences)
    std = corrected_std(differences, n_train, n_test)
    t_stat = mean / std
    p_val = t.sf(np.abs(t_stat), df)  # right-tailed t-test
    return t_stat, p_val

In [ ]:
model_1_scores = model_scores.iloc[0].values  # scores of the best model
model_2_scores = model_scores.iloc[1].values  # scores of the second-best model

differences = model_1_scores - model_2_scores

n = differences.shape[0]  # number of test sets
dof = n - 1

t_stat, p_val = compute_corrected_ttest(
    differences,
    dof,
    num_samples_train,
    num_samples_test,
)
print(f"Corrected t-statistic: {t_stat:.3f}")
print(f"Corrected p-value: {p_val:.3f}")

In [ ]:
# Test all models against the best model.
best_model_scores = model_scores.iloc[0].values

n_comparisons = model_scores.shape[0] - 1

pairwise_t_test = []

for model_i in range(1, len(model_scores)):
    model_i_scores = model_scores.iloc[model_i].values
    differences = model_i_scores - best_model_scores
    t_stat, p_val = compute_corrected_ttest(
        differences,
        dof,
        num_samples_train,
        num_samples_test,
    )

    # Implement Bonferroni correction
    p_val *= n_comparisons

    # Bonferroni can output p-values higher than 1
    p_val = 1 if p_val > 1 else p_val

    pairwise_t_test.append([
        model_scores.index[0],
        model_scores.index[model_i],
        t_stat,
        p_val,
    ])

pairwise_comp_df = pd.DataFrame(
    pairwise_t_test,
    columns=["model_1", "model_2", "t_stat", "p_val"],
).round(3)

In [ ]:
pairwise_comp_df

In [ ]:
for model_i in range(1, len(model_scores)):
    model_i_scores = model_scores.iloc[model_i].values
    differences = model_i_scores - best_model_scores

    name_i = model_scores.index[model_i]
    name_best = model_scores.index[0]

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.hist(model_i_scores, bins=30, alpha=0.5, label=name_i)
    plt.hist(best_model_scores, bins=30, alpha=0.5, label=name_best)
    plt.title(f'{name_i} vs {name_best}')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.hist(differences, bins=30, alpha=0.5)
    plt.title(f'differences {name_i} - {name_best}')
    plt.show()

## Teste

### Filtragem

In [ ]:
X_filter = np.where(X["absences"] % 2 == 0)
y_filter = np.where(y != 0)
df_filter = np.intersect1d(X_filter, y_filter)
df_filter

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X.iloc[df_filter],
    y.iloc[df_filter],
    test_size=0.2,
    random_state=42,
)

In [ ]:
pipe_Ridge_100 = Pipeline([
    ("preprocessor", preprocessing_pipeline),
    ("regressor", Ridge(alpha=100)),
])

pipe_Ridge_100.fit(X_train, y_train)

y_predict = pipe_Ridge_100.predict(X_test)
rmse = np.sqrt(np.mean((y_test - y_predict)**2))

f"RMSE = {rmse}"

### Coeficientes

In [ ]:
coefs_keys = pipe_Ridge_100[0].get_feature_names_out()
coefs_values = pipe_Ridge_100.named_steps["regressor"].coef_
size = (len(coefs_keys), len(coefs_values))

print(f"Size: {size}")

coefs = pd.Series(
    dict(
        zip(coefs_keys, coefs_values)
    )
)

coefs